In [1]:
image_paths = {
    "2018": {
        "image": "dataset/2018/m_3912112_sw_10_060_20180718.tif",
        "labels": "labels/saved_labels_2018.tif"
    },
    "2020": {
        "image": "dataset/2020/m_3912112_sw_10_060_20200709.tif",
        "labels": "labels/saved_labels_2020.tif"
    },
    "2022": {
        "image": "dataset/2022/dataset/m_3912112_sw_10_060_20220714.tif",
        "labels": "labels/saved_labels_2022.tif"
    }
}